# Play with find_trajectory_fragment

In [1]:
import os
import pickle
import random
import time
import numpy as np
import torch

In [231]:
DATASET_NAME_TO_NUM = {
    'eth': 0,
    'hotel': 1,
    'zara1': 2,
    'zara2': 3,
    'univ': 4
}

class Object(object):
    pass

args = Object()
args.dataset = 'eth5'
args.test_set = 'eth'
args.save_dir = './output/' + args.test_set + "/"
args.seq_length = 20
args.obs_length = 8
args.neighbor_thred = 10
args.batch_around_ped = 256
if not os.path.exists(args.save_dir):
    os.makedirs(args.save_dir)

self = Object()
self.args = args

setname = 'train'

In [232]:
def find_trajectory_fragment(self, trajectory, start_frame, seq_length, skip):
    """
    Starting from a full trajectory, query a trajectory fragment of
    length seq_length starting at start_frame. Replace with 0 if data
    does not exists.

    Parameters
    ----------
    trajectory : np.array
        full trajectory of an agent. Shape: len_traj * [frame,x,y]
        len_traj really depends on the considered agent, it can be
        shorter or longer than seq_length.
    start_frame : int
        Starting frame of current trajectory fragment
    seq_length : int
        Desired length of the trajectory fragment
    skip : int
        Frame_delta between two time-steps

    Returns
    -------
    return_trajec : np.array
        The trajectory fragment of shape seq_length * (x,y)
    iffull : bool
        True if the trajectory fragment is full
    ifexsitobs : bool
        cneiui
    """
    return_trajec = np.zeros((seq_length, 3))
    start_n = np.where(trajectory[:, 0] == start_frame)

    # TODO: Debug sdd
    if self.args.dataset in ['eth5', 'lyft', 'ind_v1', 'ind_v2', 'ind_v3']:
        end_frame = start_frame + seq_length * skip
        end_n = np.where(trajectory[:, 0] == end_frame)
    else:  # sdd
        end_frame = start_frame + (seq_length - 1) * skip
        end_n = np.where(trajectory[:, 0] == end_frame) + np.array([1])

    iffull = False
    ifexsitobs = False
    # if no start and end
    if start_n[0].shape[0] == 0 and end_n[0].shape[0] != 0:
        start_n = 0
        end_n = end_n[0][0]
        # if only one point, return 0's
        if end_n == 0:
            return return_trajec, iffull, ifexsitobs
    # if start and no end
    elif start_n[0].shape[0] != 0 and end_n[0].shape[0] == 0:
        start_n = start_n[0][0]
        end_n = trajectory.shape[0]
    # if no start and no end
    elif start_n[0].shape[0] == 0 and end_n[0].shape[0] == 0:
        start_n = 0
        end_n = trajectory.shape[0]
    # if start and end
    else:
        end_n = end_n[0][0]
        start_n = start_n[0][0]

    candidate_seq = trajectory[start_n:end_n]
    offset_start = int((candidate_seq[0, 0] - start_frame) // skip)

    if self.args.dataset in ['eth5', 'lyft', 'ind_v1', 'ind_v2', 'ind_v3']:
        offset_end = self.args.seq_length + int(
            (candidate_seq[-1, 0] - end_frame)//skip)
    else:  # sdd
        offset_end = self.args.seq_length + int(
            (candidate_seq[-1, 0] - end_frame)//skip) - 1

    return_trajec[offset_start:offset_end + 1, :3] = candidate_seq

    if return_trajec[self.args.obs_length - 1, 1] != 0:
        ifexsitobs = True
    if offset_end - offset_start >= seq_length - 1:
        iffull = True

    return return_trajec, iffull, ifexsitobs

In [233]:
seq_length = 20
skip = 10
start_frame = 180
end_frame = start_frame + seq_length * skip
print(start_frame)
print(end_frame)
trajectory = np.array([[skip*i,1,1] for i in range(22,52)])

180
380


In [234]:
trajectory

array([[220,   1,   1],
       [230,   1,   1],
       [240,   1,   1],
       [250,   1,   1],
       [260,   1,   1],
       [270,   1,   1],
       [280,   1,   1],
       [290,   1,   1],
       [300,   1,   1],
       [310,   1,   1],
       [320,   1,   1],
       [330,   1,   1],
       [340,   1,   1],
       [350,   1,   1],
       [360,   1,   1],
       [370,   1,   1],
       [380,   1,   1],
       [390,   1,   1],
       [400,   1,   1],
       [410,   1,   1],
       [420,   1,   1],
       [430,   1,   1],
       [440,   1,   1],
       [450,   1,   1],
       [460,   1,   1],
       [470,   1,   1],
       [480,   1,   1],
       [490,   1,   1],
       [500,   1,   1],
       [510,   1,   1]])

# Play

In [235]:

return_trajec = np.zeros((seq_length, 3))
start_n = np.where(trajectory[:, 0] == start_frame)

print(start_n)


end_n = np.where(trajectory[:, 0] == end_frame)

print(end_n)

iffull = False
ifexsitobs = False
# if no start and end
if start_n[0].shape[0] == 0 and end_n[0].shape[0] != 0:
    print("no start and end")
    start_n = 0
    end_n = end_n[0][0]
    ## if only one point, return 0's
    ## CHECK THIS
    if end_n == 0:
        print('bubu1')
        ### return return_trajec, iffull, ifexsitobs
# if start and no end
elif start_n[0].shape[0] != 0 and end_n[0].shape[0] == 0:
    print("start and no end")
    start_n = start_n[0][0]
    end_n = trajectory.shape[0]
# if no start and no end
elif start_n[0].shape[0] == 0 and end_n[0].shape[0] == 0:
    print("no start and no end")
    # if trajectory fragment included in trajectory
    if start_frame < trajectory[0, 0] and end_frame > trajectory[-1, 0]:
        print("trajectory fragment included in trajectory")
        start_n = 0
        end_n = trajectory.shape[0]
    # if trajectory fragment excluded in trajectory
    else:
        print('bubu2')
        ### return return_trajec, iffull, ifexsitobs
# if start and end
else:
    print("start and end")
    end_n = end_n[0][0]
    start_n = start_n[0][0]
    
print("start_n, end_n:", start_n, end_n)

candidate_seq = trajectory[start_n:end_n]
offset_start = int((candidate_seq[0, 0] - start_frame) // skip)


offset_end = self.args.seq_length + int((candidate_seq[-1, 0] - end_frame)//skip)

print("offsets       :", offset_start, offset_end)

return_trajec[offset_start:offset_end + 1, :3] = candidate_seq

# if return_trajec[self.args.obs_length - 1, 1] != 0:
#     ifexsitobs = True
# if offset_end - offset_start >= seq_length - 1:
#     iffull = True


(array([], dtype=int64),)
(array([16]),)
no start and end
start_n, end_n: 0 16
offsets       : 4 19


In [236]:
start_frame

180

In [237]:
trajectory

array([[220,   1,   1],
       [230,   1,   1],
       [240,   1,   1],
       [250,   1,   1],
       [260,   1,   1],
       [270,   1,   1],
       [280,   1,   1],
       [290,   1,   1],
       [300,   1,   1],
       [310,   1,   1],
       [320,   1,   1],
       [330,   1,   1],
       [340,   1,   1],
       [350,   1,   1],
       [360,   1,   1],
       [370,   1,   1],
       [380,   1,   1],
       [390,   1,   1],
       [400,   1,   1],
       [410,   1,   1],
       [420,   1,   1],
       [430,   1,   1],
       [440,   1,   1],
       [450,   1,   1],
       [460,   1,   1],
       [470,   1,   1],
       [480,   1,   1],
       [490,   1,   1],
       [500,   1,   1],
       [510,   1,   1]])

In [238]:
candidate_seq

array([[220,   1,   1],
       [230,   1,   1],
       [240,   1,   1],
       [250,   1,   1],
       [260,   1,   1],
       [270,   1,   1],
       [280,   1,   1],
       [290,   1,   1],
       [300,   1,   1],
       [310,   1,   1],
       [320,   1,   1],
       [330,   1,   1],
       [340,   1,   1],
       [350,   1,   1],
       [360,   1,   1],
       [370,   1,   1]])

In [239]:
return_trajec

array([[  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [  0.,   0.,   0.],
       [220.,   1.,   1.],
       [230.,   1.,   1.],
       [240.,   1.,   1.],
       [250.,   1.,   1.],
       [260.,   1.,   1.],
       [270.,   1.,   1.],
       [280.,   1.,   1.],
       [290.,   1.,   1.],
       [300.,   1.,   1.],
       [310.,   1.,   1.],
       [320.,   1.,   1.],
       [330.,   1.,   1.],
       [340.,   1.,   1.],
       [350.,   1.,   1.],
       [360.,   1.,   1.],
       [370.,   1.,   1.]])

In [240]:
return_trajec[self.args.obs_length - 1, 0]
    

250.0

In [241]:
self.args.obs_length

8

In [245]:
np.count_nonzero(return_trajec[:,0])

16